# Agents From Scratch

Link: [Blog post](https://www.newsletter.swirlai.com/p/building-ai-agents-from-scratch-part)

In [1]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Literal, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl

# Visualization
import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from rich.table import Table
from rich import box
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)

console = Console(theme=custom_theme)

In [16]:
from dataclasses import dataclass
import inspect
import os
from typing import _GenericAlias, Callable, get_type_hints
from urllib import request as urllib_request

import openai

### Tool Creation Utilities

In [5]:
@dataclass
class Tool:
    """A class representing a tool with a name, description, function, and parameters.

    Parameters
    ----------
    name : str
        The name of the tool
    description : str
        A description of what the tool does
    func : Callable[..., str]
        The function that implements the tool's functionality
    parameters : dict[str, dict[str, str]]
        A dictionary mapping parameter names to their descriptions and types
    """

    name: str
    description: str
    func: Callable[..., str]
    parameters: dict[str, dict[str, str]]

    def __call__(self, *args: Any, **kwds: Any) -> str:
        """Execute the tool's function with the given arguments.

        Parameters
        ----------
        *args : Any
            Positional arguments to pass to the function
        **kwds : Any
            Keyword arguments to pass to the function

        Returns
        -------
        str
            The result of executing the function
        """
        return self.func(*args, **kwds)


def parse_docstring_params(docstring: str) -> dict[str, str]:
    """Parse parameters section from a NumPy-style docstring.

    Parameters
    ----------
    docstring : str
        The docstring to parse

    Returns
    -------
    dict[str, str]
        A dictionary mapping parameter names to their descriptions
    """
    if not docstring:
        return {}

    params: dict[str, str] = {}
    lines: list[str] = docstring.split("\n")
    in_params: bool = False
    current_param: str | None = None

    for line in lines:
        line = line.strip()
        if line.startswith("Parameters:"):
            in_params = True
        elif in_params:
            if line.startswith("-") or line.startswith("*"):
                current_param = line.lstrip("- *").split(":")[0].strip()
                params[current_param] = line.lstrip("- *").split(":")[1].strip()
            elif current_param and line:
                params[current_param] += " " + line
            elif not line:
                in_params = False

    return params


def get_type_description(type_hint: Any) -> str:
    """Get a string description of a type hint.

    Parameters
    ----------
    type_hint : Any
        The type hint to describe

    Returns
    -------
    str
        A human-readable description of the type
    """
    if isinstance(type_hint, _GenericAlias):
        if type_hint._name == "Literal":
            return f"one of {type_hint.__args__}"
    return type_hint.__name__

In [6]:
# Updated version
def parse_docstring_params(docstring: str) -> dict[str, str]:
    """Parse docstring parameters into a dictionary.

    This function extracts parameter descriptions from a NumPy-style docstring
    and returns them as a dictionary mapping parameter names to their descriptions.

    Parameters
    ----------
    docstring : str
        The docstring to parse, expected to be in NumPy format.

    Returns
    -------
    dict[str, str]
        A dictionary where keys are parameter names and values are their descriptions.
        Returns an empty dict if the docstring is empty or has no parameters.
    """
    if not docstring:
        return {}

    params: dict[str, str] = {}
    lines: list[str] = docstring.split("\n")
    in_params: bool = False
    current_param: str | None = None

    for line in lines:
        line = line.strip()
        if line.startswith("Parameters"):
            in_params = True
            continue
        elif in_params:
            if (
                line.startswith("Returns")
                or line.startswith("Raises")
                or line.startswith("Notes")
                or line.startswith("Examples")
            ):
                break
            if line:
                if ":" in line:
                    current_param, description = map(str.strip, line.split(":", 1))
                    # Remove leading "-", " " or "*" from the line
                    current_param = current_param.strip("-* ")
                    params[current_param] = description
                elif current_param:
                    params[current_param] += " " + line
            else:
                in_params = False

    return params

In [7]:
# Examples
# 1: Regular
def my_func1(param1: str, param2: int) -> dict[str, str]:
    """Parse docstring parameters into a dictionary.

    Parameters:
    - param1: This is the first parameter.
    - param2: This is the second parameter.

    Returns:
    - A dictionary with parameter names as keys and their descriptions as values.
    """


# 2: NumPy Style
def my_func2(param1: str, param2: int) -> dict[str, str]:
    """Parse docstring parameters into a dictionary.

    Parameters
    ----------
    param1 : str
        This is the first parameter.
    param2 : int
        This is the second parameter.

    Returns
    -------
    dict[str, str]
        A dictionary with parameter names as keys and their descriptions as values.
    """


console.print(parse_docstring_params(my_func1.__doc__))
console.print(parse_docstring_params(my_func2.__doc__))

{'param1': 'This is the first parameter.', 'param2': 'This is the second parameter.'}

{'param1': 'str This is the first parameter.', 'param2': 'int This is the second parameter.'}

In [8]:
type_hints_dict: dict[str, Any] = get_type_hints(my_func2)
print(type_hints_dict)

print(get_type_description(type_hints_dict.get("param1")))

{'param1': <class 'str'>, 'param2': <class 'int'>, 'return': dict[str, str]}
str


<br>

### Tool Creation Decorator

In [9]:
console.print(inspect.getdoc(my_func1))

print("===== OR =====\n\n")

# OR
print(my_func1.__doc__)

Parse docstring parameters into a dictionary.

Parameters:
- param1: This is the first parameter.
- param2: This is the second parameter.

Returns:
- A dictionary with parameter names as keys and their descriptions as values.

===== OR =====


Parse docstring parameters into a dictionary.

    Parameters:
    - param1: This is the first parameter.
    - param2: This is the second parameter.

    Returns:
    - A dictionary with parameter names as keys and their descriptions as values.
    


In [15]:
print(inspect.signature(my_func2))
print(inspect.signature(my_func2).parameters.items())

(param1: str, param2: int) -> dict[str, str]
odict_items([('param1', <Parameter "param1: str">), ('param2', <Parameter "param2: int">)])


In [12]:
def tool(name: str | None = None) -> str:
    """Decorator function to create a Tool object from a function.

    Parameters
    ----------
    name : str | None, optional
        Custom name for the tool. If None, uses the function name, by default None

    Returns
    -------
    Callable
        Decorator function that creates a Tool object
    """

    def decorator(func: Callable[..., str]) -> Tool:
        """Inner decorator function that processes the decorated function.

        Parameters
        ----------
        func : Callable[..., str]
            Function to be converted into a Tool object

        Returns
        -------
        Tool
            Tool object created from the decorated function
        """
        tool_name: str = name or func.__name__
        description: str = inspect.getdoc(func) or "No description provided."

        type_hints: dict[str, Any] = get_type_hints(func)
        param_docs: dict[str, str] = parse_docstring_params(description)
        sig: inspect.Signature = inspect.signature(func)
        params: dict[str, Any] = {}

        for param_name, _ in sig.parameters.items():
            params[param_name] = {
                "type": get_type_description(type_hints.get(param_name)),
                "description": param_docs.get(param_name, "No description provided."),
            }
        return Tool(
            name=tool_name,
            description=description,
            func=func,
            parameters=params,
        )

    return decorator

### Create Currency Convertion Tool

In [36]:
@tool()
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert an amount from one currency to another using exchange rates.

    Parameters
    ----------
    amount : float
        The amount of money to convert
    from_currency : str
        The source currency code (e.g., 'USD', 'EUR', NGN, etc)
    to_currency : str
        The target currency code (e.g., 'USD', 'EUR', NGN, etc)

    Returns
    -------
    str
        A formatted string containing the conversion result or error message
    """
    try:
        url: str = f"https://api.exchangerate-api.com/v4/latest/{from_currency.upper()}"
        with urllib_request.urlopen(url) as response:
            data: dict[str, dict[str, float] | str] = json.loads(response.read())

        if "rates" not in data:
            return "Error: Could not retrieve exchange rates."

        rate: float = data["rates"].get(to_currency.upper())  # type: ignore
        if not rate:
            return f"Error: Currency conversion not supported for {to_currency}."

        converted: float = amount * rate
        return f"{amount} {from_currency} is equal to {converted:,} {to_currency}"

    except Exception as e:
        return f"Conversion error: {str(e)}"

In [37]:
convert_currency(amount=1000, from_currency="USD", to_currency="NGN")

'1000 USD is equal to 1,538,440.0 NGN'

In [29]:
import requests

from_currency: str = "USD"
url: str = f"https://api.exchangerate-api.com/v4/latest/{from_currency.upper()}"

resp = requests.get(url)
resp.json().get("rates")

In [41]:
from pydantic_settings import BaseSettings, SettingsConfigDict


class Settings(BaseSettings):
    """Settings for the application."""

    OPENAI_API_KEY: str
    model_config = SettingsConfigDict(env_file=".env", env_file_encoding="utf-8")


settings = Settings()

### Agent Class

In [42]:
class Agent:
    """A class that manages AI tools and executes queries using OpenAI's API.

    This class provides functionality to add tools, manage them, and execute
    user queries by either using the tools or providing direct responses.

    Attributes
    ----------
    client : openai.OpenAI
        The OpenAI client instance for making API calls
    tools : dict[str, Tool]
        Dictionary mapping tool names to Tool instances
    """

    def __init__(self) -> None:
        """Initialize the Agent with OpenAI client and empty tools dictionary."""
        self.client = openai.OpenAI(api_key=settings.OPENAI_API_KEY)
        self.tools: dict[str, Tool] = {}

    def add_tool(self, tool: Tool) -> None:
        """Add a tool to the agent's available tools.

        Parameters
        ----------
        tool : Tool
            The tool instance to add
        """
        self.tools[tool.name] = tool

    def get_available_tools(self) -> list[str]:
        """Get a list of available tools with their descriptions.

        Returns
        -------
        list[str]
            List of strings containing tool names and descriptions
        """
        return [f"{tool.name}: {tool.description}" for tool in self.tools.values()]

    def use_tool(self, tool_name: str, **kwargs: Any) -> Any:
        """Execute a specific tool with given arguments.

        Parameters
        ----------
        tool_name : str
            Name of the tool to execute
        **kwargs : Any
            Keyword arguments to pass to the tool

        Returns
        -------
        Any
            Result of the tool execution

        Raises
        ------
        ValueError
            If the specified tool is not found
        """
        if tool_name not in self.tools:
            raise ValueError(
                f"Tool '{tool_name}' not found. Available tools: {list(self.tools.keys())}"
            )

        tool = self.tools[tool_name]
        return tool.func(**kwargs)

    def create_system_prompt(self) -> str:
        """Create the system prompt for the AI assistant.

        Returns
        -------
        str
            Formatted system prompt containing tools and instructions
        """
        tools_json: dict[str, Any] = {
            "role": "AI Assistant",
            "capabilities": [
                "Using provided tools to help users when needed.",
                "Responding directly to user queries when no tools are needed.",
                "Planning efficient tool usage sequences.",
            ],
            "instructions": [
                "Use tools only when they are needed.",
                "If a query can be answered without tools, respond directly.",
                "When tool are needed, plan their usage efficiently to minimize tools calls.",
            ],
            "tools": [
                {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": {
                        name: {
                            "type": info["type"],
                            "description": info["description"],
                        }
                        for name, info in tool.parameters.items()
                    },
                }
                for tool in self.tools.values()
            ],
            "response_format": {
                "type": "json",
                "schema": {
                    "requires_tools": {
                        "type": "boolean",
                        "description": "whether this query requires the use of tools.",
                    },
                    "direct_response": {
                        "type": "string",
                        "description": "the response to the query if no tools are needed.",
                        "optional": True,
                    },
                    "thought": {
                        "type": "string",
                        "description": "the thought process of the AI assistant (when tools are needed).",
                        "optional": True,
                    },
                    "plan": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "steps to solve the task (when tools are needed).",
                        "optional": True,
                    },
                    "tool_calls": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "tool": {
                                    "type": "string",
                                    "description": "the name of the tool to call.",
                                },
                                "args": {
                                    "type": "string",
                                    "description": "the parameters for the tool.",
                                },
                            },
                        },
                        "description": "the tools to be called (when tools are needed).",
                        "optional": True,
                    },
                },
                "examples": [
                    {
                        "query": "Convert 100 USD to EUR",
                        "response": {
                            "requires_tools": True,
                            "thought": "I need to use the currency converter tool to convert USD to EUR.",
                            "plan": [
                                "Use the convert_currency tool to convert 100 USD to EUR",
                                "Return the result of the conversion",
                            ],
                            "tool_calls": [
                                {
                                    "tool": "convert_currency",
                                    "args": {
                                        "amount": 100,
                                        "from_currency": "USD",
                                        "to_currency": "EUR",
                                    },
                                }
                            ],
                        },
                    },
                    {
                        "query": "Convert 50 USD to NGN",
                        "response": {
                            "requires_tools": True,
                            "thought": "I need to use the currency converter tool to convert USD to NGN.",
                            "plan": [
                                "Use the convert_currency tool to convert 50 USD to NGN",
                                "Return the result of the conversion",
                            ],
                            "tool_calls": [
                                {
                                    "tool": "convert_currency",
                                    "args": {
                                        "amount": 50,
                                        "from_currency": "USD",
                                        "to_currency": "NGN",
                                    },
                                }
                            ],
                        },
                    },
                    {
                        "query": "What currency does Nigeria use?",
                        "response": {
                            "requires_tools": False,
                            "direct_response": "Nigeria uses the Nigerian Naira (NGN) as its official currency. "
                            "This is common knowledge that doesn't require using the currency conversion tool.",
                        },
                    },
                ],
            },
        }
        return f"""You're an AI assistant that helps users by providing direct answers or using tools when necessary.
    Configuration, instructions, and available tools are provided in JSON format below:

    {json.dumps(tools_json, indent=2)}

    Always respond with a JSON object following the response_format schema above. 
    Remember to use tools only when they are actually needed for the task."""

    def plan(self, user_query: str) -> dict[str, Any]:
        """Create an execution plan for the user query.

        Parameters
        ----------
        user_query : str
            The query from the user

        Returns
        -------
        dict[str, Any]
            A dictionary containing the execution plan

        Raises
        ------
        ValueError
            If the response cannot be parsed as JSON
        """
        messages = [
            {"role": "system", "content": self.create_system_prompt()},
            {"role": "user", "content": user_query},
        ]
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0,
        )
        try:
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            raise ValueError("Failed to parse response as JSON.")

    def execute(self, user_query: str) -> str:
        """Execute the user query using available tools or direct response.

        Parameters
        ----------
        user_query : str
            The query from the user

        Returns
        -------
        str
            The execution result or error message
        """
        try:
            plan = self.plan(user_query)
            if not plan.get("requires_tools", True):
                return plan["direct_response"]

            # Execute each tool sequence
            results: list[str] = []
            for tool_call in plan["tool_calls"]:
                tool_name = tool_call["tool"]
                tool_args = tool_call["args"]
                result = self.use_tool(tool_name, **tool_args)
                results.append(result)

            return (
                f"Thought: {plan['thought']}"
                f"\nPlan: {'. '.join(plan['plan'])}"
                f"\nResults: {'. '.join(results)}"
            )
        except Exception as e:
            return f"Error execting plan: {str(e)}"

### Create and run the Agent

In [51]:
agent = Agent()
agent.add_tool(convert_currency)

query_list: list[str] = [
    "Convert 100 USD to NGN",
    "What's 500 Japanese Yen in British Pounds?",
    "Which country uses Naira as its currency?",
    "What's the population of Nigeria?",
]

response_dict: dict[str, Any] = {}
for query in query_list:
    response = agent.execute(query)
    response_dict[query] = response

    print(f"Query: {query}\nResponse: {response}\n")

Query: Convert 100 USD to NGN
Response: Thought: I need to use the currency converter tool to convert USD to NGN.
Plan: Use the convert_currency tool to convert 100 USD to NGN. Return the result of the conversion
Results: 100 USD is equal to 153,844.0 NGN

Query: What's 500 Japanese Yen in British Pounds?
Response: Thought: I need to use the currency converter tool to convert Japanese Yen to British Pounds.
Plan: Use the convert_currency tool to convert 500 Japanese Yen to British Pounds. Return the result of the conversion
Results: 500 JPY is equal to 2.52 GBP

Query: Which country uses Naira as its currency?
Response: Nigeria uses the Naira (NGN) as its official currency.

Query: What's the population of Nigeria?
Response: As of 2023, the estimated population of Nigeria is over 223 million people, making it the most populous country in Africa.



In [47]:
console.print(response_dict)

{
    'Convert 100 USD to NGN': 'Thought: I need to use the currency converter tool to convert USD to NGN.\nPlan: Use
the convert_currency tool to convert 100 USD to NGN. Return the result of the conversion\nResults: 100 USD is equal
to 153,844.0 NGN',
    "What's 500 Japanese Yen in British Pounds?": 'Thought: I need to use the currency converter tool to convert 
Japanese Yen (JPY) to British Pounds (GBP).\nPlan: Use the convert_currency tool to convert 500 JPY to GBP. Return 
the result of the conversion\nResults: 500 JPY is equal to 2.52 GBP',
    'Which country uses Naira as its currency?': 'Nigeria uses the Naira (NGN) as its official currency.'
}

In [53]:
@tool()
def perform_math(first_number: float, second_number: float, operation: str) -> str:
    """Perform basic arithmetic operations (addition or subtraction) on two numbers.

    Parameters
    ----------
    first_number : float
        The first number in the operation
    second_number : float
        The second number in the operation
    operation : str
        The type of operation to perform ('add' or 'subtract')

    Returns
    -------
    str
        A formatted string containing the calculation result or error message
    """
    try:
        operation = operation.lower()
        if operation not in ["add", "subtract"]:
            return "Error: Operation must be either 'add' or 'subtract'"

        if operation == "add":
            result: float = first_number + second_number
            return f"{first_number} + {second_number} = {result:,}"
        else:
            result: float = first_number - second_number
            return f"{first_number} - {second_number} = {result:,}"

    except Exception as e:
        return f"Calculation error: {str(e)}"

In [54]:
agent = Agent()
agent.add_tool(convert_currency)
agent.add_tool(perform_math)


query_list: list[str] = [
    "Convert 100 USD to NGN",
    "What's 500 Japanese Yen in British Pounds?",
    "Which country uses Naira as its currency?",
    "What's the population of Nigeria?",
    "I bought 2 items, one for 500 Naira and the other for 1000 Naira. What's the total cost?",
]

response_dict: dict[str, Any] = {}
for query in query_list:
    response = agent.execute(query)
    response_dict[query] = response

    print(f"Query: {query}\nResponse: {response}\n")

Query: Convert 100 USD to NGN
Response: Thought: I need to use the currency converter tool to convert USD to NGN.
Plan: Use the convert_currency tool to convert 100 USD to NGN. Return the result of the conversion
Results: 100 USD is equal to 153,844.0 NGN

Query: What's 500 Japanese Yen in British Pounds?
Response: Thought: I need to use the currency converter tool to convert Japanese Yen (JPY) to British Pounds (GBP).
Plan: Use the convert_currency tool to convert 500 JPY to GBP. Return the result of the conversion
Results: 500 JPY is equal to 2.52 GBP

Query: Which country uses Naira as its currency?
Response: Nigeria uses the Naira (NGN) as its official currency.

Query: What's the population of Nigeria?
Response: As of 2023, the estimated population of Nigeria is approximately 223 million people.

Query: I bought 2 items, one for 500 Naira and the other for 1000 Naira. What's the total cost?
Response: The total cost of the two items is 500 Naira + 1000 Naira = 1500 Naira.



In [50]:
console.print(agent.get_available_tools())

[
    "convert_currency: Convert an amount from one currency to another using exchange 
rates.\n\nParameters\n----------\namount : float\n    The amount of money to convert\nfrom_currency : str\n    The 
source currency code (e.g., 'USD', 'EUR', NGN, etc)\nto_currency : str\n    The target currency code (e.g., 'USD', 
'EUR', NGN, etc)\n\nReturns\n-------\nstr\n    A formatted string containing the conversion result or error 
message"
]

In [56]:
import json
from typing import Any, Callable, TypedDict
import openai


class Tool:
    """A class that wraps a function to be used as a tool by the Agent.

    Attributes
    ----------
    name : str
        The name of the tool
    description : str
        A description of what the tool does
    func : Callable
        The function that implements the tool's functionality
    parameters : dict
        Dictionary describing the tool's parameters
    """

    def __init__(
        self,
        name: str,
        description: str,
        func: Callable,
        parameters: dict[str, dict[str, str]],
    ) -> None:
        self.name = name
        self.description = description
        self.func = func
        self.parameters = parameters


class Agent:
    """A dynamic agent class that manages multiple AI tools and executes queries.

    This class provides functionality to add multiple tools, manage them, and execute
    user queries by either using the appropriate tools or providing direct responses.

    Attributes
    ----------
    client : openai.OpenAI
        The OpenAI client instance for making API calls
    tools : dict[str, Tool]
        Dictionary mapping tool names to Tool instances
    """

    def __init__(self) -> None:
        """Initialize the Agent with OpenAI client and empty tools dictionary."""
        self.client = openai.OpenAI(api_key=settings.OPENAI_API_KEY)
        self.tools: dict[str, Tool] = {}

    def add_tool(self, tool: Tool) -> None:
        """Add a tool to the agent's available tools.

        Parameters
        ----------
        tool : Tool
            The tool instance to add
        """
        self.tools[tool.name] = tool

    def add_multiple_tools(self, tools: list[Tool]) -> None:
        """Add multiple tools to the agent at once.

        Parameters
        ----------
        tools : list[Tool]
            List of tool instances to add
        """
        for tool in tools:
            self.add_tool(tool)

    def remove_tool(self, tool_name: str) -> None:
        """Remove a tool from the agent's available tools.

        Parameters
        ----------
        tool_name : str
            Name of the tool to remove

        Raises
        ------
        KeyError
            If the tool name doesn't exist
        """
        if tool_name in self.tools:
            del self.tools[tool_name]
        else:
            raise KeyError(f"Tool '{tool_name}' not found in available tools")

    def get_tool(self, tool_name: str) -> Tool:
        """Get a specific tool by name.

        Parameters
        ----------
        tool_name : str
            Name of the tool to retrieve

        Returns
        -------
        Tool
            The requested tool instance

        Raises
        ------
        KeyError
            If the tool name doesn't exist
        """
        if tool_name in self.tools:
            return self.tools[tool_name]
        raise KeyError(f"Tool '{tool_name}' not found in available tools")

    def get_available_tools(self) -> list[str]:
        """Get a list of available tools with their descriptions.

        Returns
        -------
        list[str]
            List of strings containing tool names and descriptions
        """
        return [f"{tool.name}: {tool.description}" for tool in self.tools.values()]

    def create_system_prompt(self) -> str:
        """Create a dynamic system prompt based on available tools.

        Returns
        -------
        str
            Formatted system prompt containing all available tools and instructions
        """
        tools_json: dict[str, Any] = {
            "role": "AI Assistant",
            "capabilities": [
                "Using various tools to help users with different tasks",
                "Responding directly to user queries when no tools are needed",
                "Planning efficient tool usage sequences",
                "Combining multiple tools when needed to solve complex tasks",
            ],
            "instructions": [
                "Use appropriate tools based on the specific requirements of each query",
                "Combine multiple tools when needed to solve complex tasks",
                "If a query can be answered without tools, respond directly",
                "Plan tool usage efficiently to minimize API calls",
            ],
            "tools": [
                {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": tool.parameters,
                }
                for tool in self.tools.values()
            ],
        }

        return f"""You're a versatile AI assistant that helps users by providing direct answers or using appropriate tools.
Available tools and configuration are provided in JSON format below:

{json.dumps(tools_json, indent=2)}

For queries requiring tools, respond with JSON containing:
- requires_tools: true
- thought: your reasoning
- plan: steps to solve the task
- tool_calls: array of tool calls with name and arguments

For direct responses, respond with JSON containing:
- requires_tools: false
- direct_response: your answer

Always use the most appropriate tool(s) for the task at hand."""

    def execute(self, user_query: str) -> str:
        """Execute the user query using available tools or direct response.

        This method can handle both single and multi-tool operations, choosing
        the appropriate tools based on the query requirements.

        Parameters
        ----------
        user_query : str
            The query from the user

        Returns
        -------
        str
            The execution result or error message
        """
        try:
            plan = self.plan(user_query)

            if not plan.get("requires_tools", True):
                return plan["direct_response"]

            # Execute multiple tool calls if needed
            results: list[str] = []
            for tool_call in plan["tool_calls"]:
                tool_name = tool_call["tool"]
                tool_args = tool_call["args"]
                result = self.use_tool(tool_name, **tool_args)
                results.append(result)

            # Format output based on number of results
            if len(results) == 1:
                final_result = results[0]
            else:
                final_result = "\n".join([f"- {result}" for result in results])

            return (
                f"Thought: {plan['thought']}\n"
                f"Plan: {'. '.join(plan['plan'])}\n"
                f"Results: {final_result}"
            )
        except Exception as e:
            return f"Error executing plan: {str(e)}"


# Example tool creation functions
def create_currency_tool() -> Tool:
    """Create a Tool instance for currency conversion."""
    return Tool(
        name="convert_currency",
        description="Convert an amount from one currency to another",
        func=convert_currency,
        parameters={
            "amount": {"type": "float", "description": "The amount to convert"},
            "from_currency": {
                "type": "string",
                "description": "Source currency code (e.g., USD, EUR)",
            },
            "to_currency": {
                "type": "string",
                "description": "Target currency code (e.g., USD, EUR)",
            },
        },
    )


def create_math_tool() -> Tool:
    """Create a Tool instance for math operations."""
    return Tool(
        name="perform_math",
        description="Perform basic arithmetic operations",
        func=perform_math,
        parameters={
            "first_number": {
                "type": "float",
                "description": "First number in the operation",
            },
            "second_number": {
                "type": "float",
                "description": "Second number in the operation",
            },
            "operation": {
                "type": "string",
                "description": "Operation to perform (add or subtract)",
            },
        },
    )


# Example usage
def setup_agent() -> Agent:
    """Create and configure an Agent with multiple tools.

    Returns
    -------
    Agent
        A configured Agent instance with multiple tools
    """
    agent = Agent()

    # Create tools
    currency_tool = create_currency_tool()
    math_tool = create_math_tool()

    # Add multiple tools at once
    agent.add_multiple_tools([currency_tool, math_tool])

    return agent

In [58]:
import json
from typing import Any, Callable, TypedDict
import openai


class Tool:
    """A class that wraps a function to be used as a tool by the Agent.

    Attributes
    ----------
    name : str
        The name of the tool
    description : str
        A description of what the tool does
    func : Callable
        The function that implements the tool's functionality
    parameters : dict
        Dictionary describing the tool's parameters
    """

    def __init__(
        self,
        name: str,
        description: str,
        func: Callable,
        parameters: dict[str, dict[str, str]],
    ) -> None:
        self.name = name
        self.description = description
        self.func = func
        self.parameters = parameters


class Agent:
    """A dynamic agent class that manages multiple AI tools and executes queries.

    This class provides functionality to add multiple tools, manage them, and execute
    user queries by either using the appropriate tools or providing direct responses.

    Attributes
    ----------
    client : openai.OpenAI
        The OpenAI client instance for making API calls
    tools : dict[str, Tool]
        Dictionary mapping tool names to Tool instances
    """

    def __init__(self) -> None:
        """Initialize the Agent with OpenAI client and empty tools dictionary."""
        self.client = openai.OpenAI(api_key=settings.OPENAI_API_KEY)
        self.tools: dict[str, Tool] = {}

    def add_tool(self, tool: Tool) -> None:
        """Add a tool to the agent's available tools."""
        self.tools[tool.name] = tool

    def add_multiple_tools(self, tools: list[Tool]) -> None:
        """Add multiple tools to the agent at once."""
        for tool in tools:
            self.add_tool(tool)

    def remove_tool(self, tool_name: str) -> None:
        """Remove a tool from the agent's available tools."""
        if tool_name in self.tools:
            del self.tools[tool_name]
        else:
            raise KeyError(f"Tool '{tool_name}' not found in available tools")

    def get_tool(self, tool_name: str) -> Tool:
        """Get a specific tool by name."""
        if tool_name in self.tools:
            return self.tools[tool_name]
        raise KeyError(f"Tool '{tool_name}' not found in available tools")

    def get_available_tools(self) -> list[str]:
        """Get a list of available tools with their descriptions."""
        return [f"{tool.name}: {tool.description}" for tool in self.tools.values()]

    def use_tool(self, tool_name: str, **kwargs: Any) -> Any:
        """Execute a specific tool with given arguments."""
        if tool_name not in self.tools:
            raise ValueError(
                f"Tool '{tool_name}' not found. Available tools: {list(self.tools.keys())}"
            )

        tool = self.tools[tool_name]
        return tool.func(**kwargs)

    def create_system_prompt(self) -> str:
        """Create a dynamic system prompt based on available tools."""
        tools_json: dict[str, Any] = {
            "role": "AI Assistant",
            "capabilities": [
                "Using various tools to help users with different tasks",
                "Responding directly to user queries when no tools are needed",
                "Planning efficient tool usage sequences",
                "Combining multiple tools when needed to solve complex tasks",
            ],
            "instructions": [
                "Use appropriate tools based on the specific requirements of each query",
                "Combine multiple tools when needed to solve complex tasks",
                "If a query can be answered without tools, respond directly",
                "Plan tool usage efficiently to minimize API calls",
            ],
            "tools": [
                {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": tool.parameters,
                }
                for tool in self.tools.values()
            ],
            "response_format": {
                "type": "json",
                "schema": {
                    "requires_tools": {
                        "type": "boolean",
                        "description": "whether this query requires the use of tools",
                    },
                    "direct_response": {
                        "type": "string",
                        "description": "the response to the query if no tools are needed",
                        "optional": True,
                    },
                    "thought": {
                        "type": "string",
                        "description": "the thought process of the AI assistant (when tools are needed)",
                        "optional": True,
                    },
                    "plan": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "steps to solve the task (when tools are needed)",
                        "optional": True,
                    },
                    "tool_calls": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "tool": {
                                    "type": "string",
                                    "description": "the name of the tool to call",
                                },
                                "args": {
                                    "type": "object",
                                    "description": "the parameters for the tool",
                                },
                            },
                        },
                        "description": "the tools to be called (when tools are needed)",
                        "optional": True,
                    },
                },
            },
        }

        return f"""You're a versatile AI assistant that helps users by providing direct answers or using appropriate tools.
Available tools and configuration are provided in JSON format below:

{json.dumps(tools_json, indent=2)}

For queries requiring tools, respond with JSON containing:
- requires_tools: true
- thought: your reasoning
- plan: steps to solve the task
- tool_calls: array of tool calls with name and arguments

For direct responses, respond with JSON containing:
- requires_tools: false
- direct_response: your answer

Always use the most appropriate tool(s) for the task at hand."""

    def plan(self, user_query: str) -> dict[str, Any]:
        """Create an execution plan for the user query.

        Parameters
        ----------
        user_query : str
            The query from the user

        Returns
        -------
        dict[str, Any]
            A dictionary containing the execution plan

        Raises
        ------
        ValueError
            If the response cannot be parsed as JSON
        """
        messages = [
            {"role": "system", "content": self.create_system_prompt()},
            {"role": "user", "content": user_query},
        ]
        response = self.client.chat.completions.create(
            model="gpt-4",  # Changed from gpt-4o-mini to gpt-4
            messages=messages,
            temperature=0,
        )
        try:
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            raise ValueError("Failed to parse response as JSON.")

    def execute(self, user_query: str) -> str:
        """Execute the user query using available tools or direct response."""
        try:
            plan = self.plan(user_query)

            if not plan.get("requires_tools", True):
                return plan["direct_response"]

            # Execute multiple tool calls if needed
            results: list[str] = []
            for tool_call in plan["tool_calls"]:
                tool_name = tool_call["tool"]
                tool_args = tool_call["args"]
                result = self.use_tool(tool_name, **tool_args)
                results.append(result)

            # Format output based on number of results
            if len(results) == 1:
                final_result = results[0]
            else:
                final_result = "\n".join([f"- {result}" for result in results])

            return (
                f"Thought: {plan['thought']}\n"
                f"Plan: {'. '.join(plan['plan'])}\n"
                f"Results: {final_result}"
            )
        except Exception as e:
            return f"Error executing plan: {str(e)}"


# Example tool creation functions
def create_currency_tool() -> Tool:
    """Create a Tool instance for currency conversion."""
    return Tool(
        name="convert_currency",
        description="Convert an amount from one currency to another",
        func=convert_currency,
        parameters={
            "amount": {"type": "float", "description": "The amount to convert"},
            "from_currency": {
                "type": "string",
                "description": "Source currency code (e.g., USD, EUR)",
            },
            "to_currency": {
                "type": "string",
                "description": "Target currency code (e.g., USD, EUR)",
            },
        },
    )


def create_math_tool() -> Tool:
    """Create a Tool instance for math operations."""
    return Tool(
        name="perform_math",
        description="Perform basic arithmetic operations",
        func=perform_math,
        parameters={
            "first_number": {
                "type": "float",
                "description": "First number in the operation",
            },
            "second_number": {
                "type": "float",
                "description": "Second number in the operation",
            },
            "operation": {
                "type": "string",
                "description": "Operation to perform (add or subtract)",
            },
        },
    )


# Example usage
def setup_agent() -> Agent:
    """Create and configure an Agent with multiple tools."""
    agent = Agent()

    # Create tools
    currency_tool = create_currency_tool()
    math_tool = create_math_tool()

    # Add multiple tools at once
    agent.add_multiple_tools([currency_tool, math_tool])

    return agent

In [59]:
# Create and test the agent
agent = setup_agent()

# Test different types of queries
print(agent.execute("Convert 100 USD to EUR"))
print(agent.execute("Add 15.5 and 7.3"))
print(agent.execute("What is the capital of France?"))  # Direct response

Thought: The user wants to convert a certain amount of money from one currency to another. This task requires the use of the 'convert_currency' tool.
Plan: Use the 'convert_currency' tool to convert the amount from USD to EUR
Results: 100 USD is equal to 96.0 EUR
Thought: The user wants to perform an arithmetic operation. I have a tool for this task: 'perform_math'.
Plan: Use the 'perform_math' tool to add 15.5 and 7.3
Results: 15.5 + 7.3 = 22.8
The capital of France is Paris.
